In [2]:
import pandas as pd

In [3]:
# loading the dataset
# the dataset was scraped from https://www.enforcementtracker.com/
# and represents a public list of fines that 
# data protection authorities within the EU have imposed under the EU General Data Protection Regulation (GDPR, DSGVO).
df = pd.read_csv("./dataset.csv")

In [4]:
df

,country,agency,date_of_decision,fine_amount_euro,controller_processor,sector,quoted_articles,type,summary,links
0,IRELAND,Data Protection Authority of Ireland,2024-09-27,"91,000,000",Meta Platforms Ireland Limited,"Media, Telecoms and Broadcasting","Art. 5 (1) f) GDPR, Art. 32 (1) GDPR, Art. 33 ...",Insufficient technical and organisational meas...,The Irish DPA (DPC) has imposed a fine of EUR ...,https://www.dataprotection.ie/en/news-media/pr...
1,NORWAY,Norwegian Supervisory Authority (Datatilsynet),2024-09-06,"20,900",Eidskog municipality,Public Sector and Education,Art. 6 GDPR,Insufficient legal basis for data processing,"The Norwegian DPA imposed a fine of EUR 20,900...",https://www.datatilsynet.no/regelverk-og-verkt...
2,NORWAY,Norwegian Supervisory Authority (Datatilsynet),2024-09-04,"12,700",University of Agder,Public Sector and Education,"Art. 32 GDPR, Art. 24 GDPR",Insufficient technical and organisational meas...,The Norwegian DPA has fined the University of ...,https://www.datatilsynet.no/aktuelt/aktuelle-n...
3,POLAND,Polish National Personal Data Protection Offic...,2024-09-02,"19,800",National Prosecutor's Office,Public Sector and Education,"Art. 6 (1) GDPR, Art. 9 (1) GDPR, Art. 33 (1) ...",Insufficient legal basis for data processing,"The Polish DPA has imposed a fine of EUR 19,80...",https://www.uodo.gov.pl/decyzje/DKN.5131.33.20...
4,SWEDEN,Data Protection Authority of Sweden (Integrite...,2024-08-29,"3,200,000",Apoteket AB.,Health Care,Art. 32 (1) GDPR,Insufficient technical and organisational meas...,The Swedish DPA has imposed a fine of EUR 3.2 ...,https://www.imy.se/globalassets/dokument/beslu...
...,...,...,...,...,...,...,...,...,...,...
2557,GERMANY,Data Protection Authority of Rhineland-Palatinate,Unknown,50,Unknown,Individuals and Private Associations,Art. 6 GDPR,Insufficient legal basis for data processing,Unlawful use of a dashcam,https://fragdenstaat.de/anfrage/einsatz-von-da...
2558,CZECH REPUBLIC,Czech Data Protection Auhtority (UOOU),Unknown,"3,140","UniCredit Bank Czech Republic and Slovakia, a.s.","Finance, Insurance and Consulting",Art. 6 GDPR,Insufficient legal basis for data processing,The bank established a personal bank account f...,https://www.uoou.cz/kontrola-zpracovani-osobni...
2559,CZECH REPUBLIC,Czech Data Protection Auhtority (UOOU),Unknown,980,Individual entrepreneur - no further details p...,Individuals and Private Associations,Art. 32 GDPR,Insufficient technical and organisational meas...,The operator of an online game was exposed to ...,https://www.uoou.cz/kontrola-zabezpeceni-osobn...
2560,CZECH REPUBLIC,Czech Data Protection Auhtority (UOOU),Unknown,588,Alza.cz a.s.,Industry and Commerce,"Art. 6 GDPR, Art. 7 GDPR",Insufficient legal basis for data processing,The company obtained a copy of photographic ID...,https://www.uoou.cz/kontrola-zpracovani-osobni...


In [5]:
df.columns

Index(['country', 'agency', 'date_of_decision', 'fine_amount_euro',
       'controller_processor', 'sector', 'quoted_articles', 'type', 'summary',
       'links'],
      dtype='object')

In [ ]:
for col in df.columns:
    print(df[col].value_counts())
    input()

country
SPAIN              932
ITALY              396
GERMANY            205
ROMANIA            193
POLAND              82
HUNGARY             71
GREECE              71
FRANCE              65
NORWAY              55
SWEDEN              51
CYPRUS              47
BELGIUM             43
LUXEMBOURG          32
CROATIA             29
IRELAND             29
THE NETHERLANDS     29
AUSTRIA             27
CZECH REPUBLIC      27
DENMARK             26
BULGARIA            25
ICELAND             21
FINLAND             21
UNITED KINGDOM      18
MALTA               14
LITHUANIA           13
LATVIA              10
SLOVAKIA             9
ESTONIA              9
PORTUGAL             7
ISLE OF MAN          4
LIECHTENSTEIN        1
Name: count, dtype: int64


agency
Spanish Data Protection Authority (aepd)                                                  932
Italian Data Protection Authority (Garante)                                               396
Romanian National Supervisory Authority for Personal Data Processing (ANSPDCP)            193
Polish National Personal Data Protection Office (UODO)                                     81
Hellenic Data Protection Authority (HDPA)                                                  71
Hungarian National Authority for Data Protection and the Freedom of Information (NAIH)     71
French Data Protection Authority (CNIL)                                                    65
Norwegian Supervisory Authority (Datatilsynet)                                             55
Data Protection Authority of Hamburg                                                       47
Cypriot Data Protection Commissioner                                                       47
Belgian Data Protection Authority (APD)              

date_of_decision
2022          70
2021          54
2020          44
2023          38
2019          24
              ..
2018-12-20     1
2019-01-10     1
2019-01-17     1
2019-01-21     1
2019-02-08     1
Name: count, Length: 936, dtype: int64


fine_amount_euro
2,000      158
3,000      133
5,000      131
10,000     114
1,000      105
          ... 
18,630       1
1,715        1
4,290        1
8,575        1
130,000      1
Name: count, Length: 481, dtype: int64


controller_processor
Private individual                           184
Unknown                                      127
Company                                       74
Vodafone España, S.A.U.                       67
Private Individual                            24
                                            ... 
TECHPUMP SOLUTIONS S.L.                        1
Setúbal municipality                           1
Portuguese National Statistical Institute      1
Rapido Finance, S.L.                           1
I.S.P.R.O.                                     1
Name: count, Length: 1644, dtype: int64


sector
Industry and Commerce                   512
Individuals and Private Associations    359
Media, Telecoms and Broadcasting        320
Public Sector and Education             274
Finance, Insurance and Consulting       247
Health Care                             243
Not assigned                            164
Employment                              161
Transportation and Energy               126
Accomodation and Hospitalty              85
Real Estate                              71
Name: count, dtype: int64


quoted_articles
Art. 5 (1) c) GDPR                                                                                                           160
Art. 6 (1) GDPR                                                                                                              153
Art. 5 GDPR, Art. 6 GDPR                                                                                                     145
Art. 13 GDPR                                                                                                                 110
Art. 32 GDPR                                                                                                                  99
                                                                                                                            ... 
Art. 5 (1) a), b) GDPR, Art. 6 (1) GDPR, Art. 7 (2) GDPR, Art. 12 (1) GDPR, Art. 13 GDPR                                       1
Art. 5 (1) e) GDPR, Art. 32 GDPR                                                 

type
Insufficient legal basis for data processing                                         774
Non-compliance with general data processing principles                               658
Insufficient technical and organisational measures to ensure information security    444
Insufficient fulfilment of data subjects rights                                      234
Insufficient fulfilment of information obligations                                   208
Insufficient cooperation with supervisory authority                                  134
Insufficient fulfilment of data breach notification obligations                       50
Insufficient involvement of data protection officer                                   24
Unknown                                                                               24
Insufficient data processing agreement                                                12
Name: count, dtype: int64


summary
The Estonian DPA (Andmekaitse Inspektsioon) fined three online pharmacies EUR 100,000 each for processing personal data without the consent of the data subjects. The data in question are prescriptions for medicines of the data subjects. Third parties were able to view another person's current prescriptions in the e-pharmacy environment without their consent, based only on access to their personal identification code. The DPA highlighted that while it must be possible to purchase prescription drugs for other people, it is the responsibility of the company to ensure that the processing of the personal data required for this purpose only takes place with the consent of the data subjects. The confirmation of another person that they may access the data, however, does not correspond to the voluntary consent of the prescription holder, since the e-pharmacy cannot check whether and for what purpose the consent was given and whether it was given voluntarily.    6
Failure to respond to 

links
https://datenschutz-hamburg.de/assets/pdf/29._taetigkeitsbericht_datenschutz_2020.PDF                                                                                                         15
https://www.zaftda.de/tb-bundeslaender/bremen/832-5-tb-dsgvo-lfd-bremen-2022-o-drs-nr-vom-24-03-2023/file                                                                                     13
https://www.zaftda.de/tb-bundeslaender/berlin/802-tb-lfd-berlin-2021-ohne-drs-nr-vom-24-05-2022/file                                                                                          13
https://www.cnil.fr/fr/les-sanctions-prononcees-par-la-cnil                                                                                                                                   12
https://azop.hr/izreceno-novih-10-upravnih-novcanih-kazni/                                                                                                                                    10
                             